In [26]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import os.path
import numpy as np

# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [27]:
tweets = pd.read_excel("tweets_Game Marvel's Spider-Man - PS4_201809101603.xlsx") #lê o excel
lista_lixo = [',','@','/',':',';','rt','r$','\n','\(','\)','#','\?','-','"','\[','\]'] #lista com o que queremos remover dos tweets

for e in lista_lixo: #Remove o 'Lixo'
    tweets.Treinamento = tweets.Treinamento.replace(e,'',regex=True)
tweets.head(10)

Treinamento  Relevancia
0  game marvel's spiderman  ps4 caão submarino  r...           0
1   ignbrasil o melhor game do homemaranha em dez...           1
2  apesar de todas as polêmicas envolvendo reclam...           1
3   promobitoficial promoção ofea  prévenda jogo ...           0
4   ignbrasil o melhor game do homemaranha em dez...           1
5   ignbrasil o melhor game do homemaranha em dez...           1
6  declare sua independência com o maravilhoso sp...           0
7  o pablomiyazawa conversou com a diretora de ae...           0
8   ignbrasil o melhor game do homemaranha em dez...           1
9   ignbrasil o melhor game do homemaranha em dez...           1

In [28]:
strain_col = pd.Series(np.sum(tweets.Treinamento + " ").split()) #Separa as palavras em strings
relevs_col = tweets.Relevancia #Cria uma coluna com as relevâncias 
strain_col.value_counts().head(10) #Imprime e conta palavras iguais

de        273
à         237
um        165
o         163
jogo      157
no        152
pra       140
spider    140
os        133
pedido    131
dtype: int64

In [29]:
prob = relevs_col.value_counts()
relevs_col.head(10)

0    0
1    1
2    1
3    0
4    1
5    1
6    0
7    0
8    1
9    1
Name: Relevancia, dtype: int64

In [30]:
prob_relevante = prob[1]/len(relevs_col) #Calcula a provabilidade de uma palavra ser relevante entre todas as palavras do DataFrame
prob_irrelevante = prob[0]/len(relevs_col) #Calcula a provabilidade de uma palavra ser irrelevante entre todas as palavras do DataFrame
print(int(prob_relevante * 100),'%', int(prob_irrelevante * 100),'%')


73 % 26 %


In [31]:
tweets_relevantes = tweets[tweets.Relevancia == 1] #Separa os tweets relevantes
tweets_irrelevantes = tweets[tweets.Relevancia == 0] #Separa os tweets irrelevantes
Quantidade_em_relev =  pd.Series(np.sum(tweets_relevantes.Treinamento + " ").split())
Quantidade_em_relev.value_counts().head(20) #Imprime uma coluna com as quantidades de cada plavra dentro do grupo de relevantes

à                  236
de                 218
um                 150
jogo               143
no                 141
casamento          128
pra                127
pedido             127
os                 124
insomniac          122
meu                122
dele               119
namorada           119
pediu              119
deus               118
como…              118
colocar            118
rafafernandes64    118
colocaram          118
caras              118
dtype: int64

In [32]:
Quantidade_em_irrelev =  pd.Series(np.sum(tweets_irrelevantes.Treinamento + " ").split())
Quantidade_em_irrelev.value_counts().head(20) #Imprime uma coluna com as quantidades de cada plavra dentro do grupo de relevantes

o            71
spider       58
de           55
man          45
do           31
a            30
e            29
ps4          28
que          25
é            19
eu           19
spiderman    19
marvel's     16
um           15
não          14
jogo         14
com          14
game         14
mas          13
pra          13
dtype: int64

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [39]:
Verifica_data = pd.read_excel('tweets_spider man_201809182225.xlsx')
for e in lista_lixo: #Remove o 'Lixo'
    Verifica_data.Treinamento = Verifica_data.Treinamento.replace(e,'',regex=True)

Treinamento
0               o jogo do spider man eu quero jogaaaaa
1    i liked a youtube video httpst.coswf0s6msea o ...
2    zerei o modo história do spider man e confesso...
3    gostei de um vídeo youtube httpst.coi9jxavdrba...
4    gostei de um vídeo youtube httpst.cotx4u5s3txj...
5      spider man ta tao lindo q ja joguei 50% do jogo
6    agora não preciso dormir pq tenho spider man h...
7    gostei de um vídeo youtube httpst.coqgtlbkjb3o...
8    gostei de um vídeo youtube httpst.cobjwcox2ky0...
9    adicionei um vídeo a uma playlist youtube http...
10   o vídeo do o miranha jogando spider man ta mui...
11   início da série spider man no canal httpst.com...
12   i liked a youtube video httpst.cohh5ir5541v im...
13    unlockscreen ☆spider man• se salvoufav se gos...
14   gostei de um vídeo youtube httpst.coh9pvd0buhd...
15                          no véi já zerei spider man
16   nunomarmeleiroo msm... fico pelo ultimate spid...
17   gostei de um vídeo youtube httpst.cooier0ug1cu...
18   gostei de um vídeo youtube httpst.coi9tnzfjtfy...
19   snowlzlol eu concordo com você o spider man ro...
20   kyobr e o choro não para da comunidade tóxica ...
21   gostei de um vídeo youtube httpst.codtiezjfxli...
22   gostei de um vídeo youtube httpst.conk59gnb1vo...
23   i liked a youtube video httpst.cotb2yxeifzo o ...
24    unlockscreen ☆spider man• se salvoufav se gos...
25   i liked a youtube video httpst.constjfksa8g o ...
26   gostei de um vídeo youtube httpst.coadtnnuqvwk...
27    unlockscreen ☆spider man• se salvoufav se gos...
28   gostei de um vídeo youtube httpst.cozclg4qelxs...
29   seventydevils oi compra spider man e me chama ...
..                                                 ...
270  pleasuretokilll simtava dando penaherói favori...
271   andiebambii quanto é que tá o jogo do spider ...
272  gostei de um vídeo youtube httpst.co5kedegxzcu...
273       meups4 subindo e subindo httpst.cotyaqx9ubza
274  gostei de um vídeo do youtube httpst.cosop3uag...
275  gostei de um vídeo youtube httpst.cosiqpjt03sr...
276  como assim o spider man acabou assim wrf vou s...
277  obvio lançaram uma  semana depois de spider ma...
278  gostei de um vídeo youtube httpst.cokpx9svcjn6...
279   ondetaolucas stan lee apareceu até no novo jo...
280   unlockscreen ☆spider man• se salvoufav se gos...
281   unlockscreen ☆spider man• se salvoufav se gos...
282  falando em comprar coisas pra essa coisinha co...
283   unlockscreen ☆spider man• se salvoufav se gos...
284  leila_catu amiga sim! mas só se foi alguma pro...
285  diretor da insomniac explica ausência do traje...
286  gostei de um vídeo youtube httpst.colubeixfq4p...
287                 vou agora acabar o spider man... 😭
288  pedrodbr aehh zerei spider man e sinceramente ...
289  gostei de um vídeo do youtube httpst.coo5r4lc0...
290  gostei de um vídeo youtube httpst.coyxadlkyj2d...
291  gostei de um vídeo youtube httpst.co8vtcv4jfxx...
292  só queria virar essa noite jogando spider man ...
293  gostei de um vídeo youtube httpst.coutrxbu4dkk...
294  gostei de um vídeo youtube httpst.coxnecaxjcgo...
295  mano shaw mendes é igualzin o spider man velho...
296  adicionei um vídeo a uma playlist youtube http...
297  e eu q premeditei o nome do drone do spider ma...
298  stan lee apareceu até no novo jogo do spider m...
299  gostei de um vídeo youtube httpst.colsowjfu8qg...

[300 rows x 1 columns]

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
